In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
train.head()

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,The room was kind of clean but had a VERY stro...,Edge,Mobile,not happy
1,id10327,I stayed at the Crown Plaza April -- - April -...,Internet Explorer,Mobile,not happy
2,id10328,I booked this hotel through Hotwire at the low...,Mozilla,Tablet,not happy
3,id10329,Stayed here with husband and sons on the way t...,InternetExplorer,Desktop,happy
4,id10330,My girlfriends and I stayed here to celebrate ...,Edge,Tablet,not happy


In [4]:
for df1 in train,test:  
    df1.loc[:, 'Description'] = df1['Description'].str.lower()
train.head(10)    

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,the room was kind of clean but had a very stro...,Edge,Mobile,not happy
1,id10327,i stayed at the crown plaza april -- - april -...,Internet Explorer,Mobile,not happy
2,id10328,i booked this hotel through hotwire at the low...,Mozilla,Tablet,not happy
3,id10329,stayed here with husband and sons on the way t...,InternetExplorer,Desktop,happy
4,id10330,my girlfriends and i stayed here to celebrate ...,Edge,Tablet,not happy
5,id10331,we had - rooms. one was very nice and clearly ...,InternetExplorer,Desktop,happy
6,id10332,my husband and i have stayed in this hotel a f...,Firefox,Tablet,not happy
7,id10333,my wife & i stayed in this glorious city a whi...,Google Chrome,Mobile,happy
8,id10334,my boyfriend and i stayed at the fairmont on a...,Internet Explorer,Desktop,happy
9,id10335,"wonderful staff, great location, but it was de...",Chrome,Tablet,not happy


In [5]:
import unicodedata
import sys

tbl = dict.fromkeys(i for i in range(sys.maxunicode)
                      if unicodedata.category(chr(i)).startswith('P'))
def remove_punctuation(text):
    return text.translate(tbl)

test['Description']=test['Description'].apply( lambda x: remove_punctuation(x))
train['Description']=train['Description'].apply( lambda x: remove_punctuation(x))

In [6]:
train=train.drop(['Browser_Used','Device_Used','User_ID'],axis=1)
test=test.drop(['Browser_Used','Device_Used'],axis=1)


In [7]:
import nltk.stem
from nltk.stem import WordNetLemmatizer

class WordNetStemmer(WordNetLemmatizer):
    def stem(self,word,pos=u'n'):
        return self.lemmatize(word,pos)

In [8]:

steamer=WordNetStemmer()
train['Description']=pd.DataFrame([steamer.stem(w) for w in train['Description']],columns=['Descripton'])
test['Description']=pd.DataFrame([steamer.stem(w) for w in test['Description']],columns=['Descripton'])

print(train)

                                             Description Is_Response
0      the room was kind of clean but had a very stro...   not happy
1      i stayed at the crown plaza april   april   th...   not happy
2      i booked this hotel through hotwire at the low...   not happy
3      stayed here with husband and sons on the way t...       happy
4      my girlfriends and i stayed here to celebrate ...   not happy
5      we had  rooms one was very nice and clearly ha...       happy
6      my husband and i have stayed in this hotel a f...   not happy
7      my wife  i stayed in this glorious city a whil...       happy
8      my boyfriend and i stayed at the fairmont on a...       happy
9      wonderful staff great location but it was defi...   not happy
10     steps off times square nice rooms stayed  nigh...       happy
11     me the wife and  kids stayed here on valentine...       happy
12     we stay at the jolly madison over the xmas per...   not happy
13     i highly recommend the hawt

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=2,max_df=5,stop_words='english')
print(vectorizer)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=5, max_features=None, min_df=2,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)


In [10]:
vectorizer.fit(train['Description'])


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=5, max_features=None, min_df=2,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [11]:

X_train = vectorizer.transform(train['Description'])

X_train

<38932x13770 sparse matrix of type '<class 'numpy.float64'>'
	with 39244 stored elements in Compressed Sparse Row format>

In [12]:
feature_set = X_train
response=['not happy','happy']
for key,value in zip(response,(0,1)):
        train.loc[train['Is_Response'] == key, 'Is_Response'] = int(value)
train.Is_Response=train.Is_Response.astype(int) 
           

In [13]:
X_test = vectorizer.transform(test['Description'])
print(X_test)

  (0, 10605)	0.700545289988
  (0, 2066)	0.713607943254
  (3, 12206)	0.724559646301
  (3, 10133)	0.68921210012
  (8, 7284)	1.0
  (9, 11275)	1.0
  (10, 7188)	1.0
  (13, 358)	1.0
  (17, 13365)	0.714896045841
  (17, 9204)	0.699230751355
  (18, 606)	1.0
  (20, 12812)	1.0
  (21, 9606)	1.0
  (23, 3980)	1.0
  (25, 9778)	1.0
  (28, 11473)	0.707106781187
  (28, 6890)	0.707106781187
  (29, 11418)	1.0
  (34, 10135)	0.730883995134
  (34, 7017)	0.682501711102
  (36, 11023)	0.410929724294
  (36, 7432)	0.821859448587
  (36, 6978)	0.394567875606
  (37, 5986)	1.0
  (38, 13437)	0.697190260174
  :	:
  (29367, 8523)	1.0
  (29368, 13387)	1.0
  (29370, 11565)	0.488192890311
  (29370, 7758)	0.513230786094
  (29370, 5667)	0.499130174431
  (29370, 2943)	0.499130174431
  (29371, 13145)	0.338490947063
  (29371, 11847)	0.662147400644
  (29371, 9341)	0.325013368464
  (29371, 6188)	0.338490947063
  (29371, 4902)	0.34805344126
  (29371, 4149)	0.325013368464
  (29375, 6182)	1.0
  (29379, 11446)	1.0
  (29381, 3909)	1.0

In [14]:
from sklearn.cross_validation import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X_train,train['Is_Response'])

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [25]:
from sklearn.ensemble import ExtraTreesClassifier

clf = ExtraTreesClassifier(n_estimators=200,n_jobs=-1, random_state=0)
clf.fit(x_train,y_train)


ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [26]:
from sklearn.metrics import accuracy_score

prediction = clf.predict(x_test)
accuracy_score(y_test,prediction)

0.67707798212267545

In [29]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
                         algorithm="SAMME",
                         n_estimators=200)

bdt.fit(x_train, y_train)


AdaBoostClassifier(algorithm='SAMME',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=1.0, n_estimators=200, random_state=None)

In [30]:
prediction = bdt.predict(x_test)
accuracy_score(y_test,prediction)

0.67718072536730711

In [23]:
from xgboost import XGBClassifier

# fit model no training data
model = XGBClassifier(n_estimators=200)
model.fit(x_train,y_train)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=200,
       n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=-1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [24]:
from sklearn.metrics import accuracy_score

prediction = model.predict(x_test)
accuracy_score(y_test,prediction)

0.67718072536730711

In [52]:
#submission=pd.DataFrame({'User_ID':test['User_ID']})
#submission['Is_Response']=pd.DataFrame(prediction)
#for key, value in zip((0,1),response):
#        submission.loc[submission['Is_Response'] == key, "Is_Response"] = value
#submission    

,User_ID,Is_Response
0,id80132,happy
1,id80133,happy
2,id80134,happy
3,id80135,happy
4,id80136,happy
5,id80137,happy
6,id80138,happy
7,id80139,happy
8,id80140,happy
9,id80141,happy


In [53]:
#submission.to_csv('submission2.csv',index=False)